# Advertising Effectiveness

In [ ]:
# Enable the commands below when running this program on Google Colab.
# !pip install arviz==0.7
# !pip install pymc3==3.8
# !pip install Theano==1.0.4

import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

import pymc3 as pm

import math≈

plt.style.use('seaborn-darkgrid')
np.set_printoptions(precision=3)
pd.set_option('display.precision', 3)

In [ ]:
columns = ['Not purchased', 'Purchased']
index = ['Watched CM', 'Not watched']
data = pd.DataFrame([[33, 18], [84, 23]], columns=columns, index=index)
display(data)
N = data.sum().sum()
print(N)

observed = [data['Not purchased']['Watched CM'],
            data['Purchased']['Watched CM'],
            data['Not purchased']['Not watched'],
            data['Purchased']['Not watched']]
print(observed)

## Bayesian analysis

In [ ]:
with pm.Model() as model:
    # Prior distribution
    p_ = pm.Uniform('p_', 0, 1, shape=4)
    p = pm.Deterministic('p', p_ / pm.math.sum(p_))

    # Likelihood
    y_pred = pm.Multinomial('y_pred', n=N, p=p, observed=observed)

    # Marginal probability
    p1d = p[0] + p[1]
    p2d = p[2] + p[3]

    pd1 = p[0] + p[2]
    pd2 = p[1] + p[3]

    # Pearson's residual
    pp = [p1d * pd1, p1d * pd2,
        p2d * pd1, p2d * pd2]
    e = pm.Deterministic('e', (p - pp) / pm.math.sqrt(pp))

    # Cramer's association coefficient
    V = pm.Deterministic('V', pm.math.sqrt(pm.math.sum(e**2)))

    trace = pm.sample(21000, chains=5)

In [ ]:
chain = trace[1000:]
pm.traceplot(chain)
plt.show()

In [ ]:
pm.summary(chain, var_names=['p', 'e', 'V'])

### Independence and association

In [ ]:
plt.boxplot(
    [chain['e'][:,0],
     chain['e'][:,1],
     chain['e'][:,2],
     chain['e'][:,3]],
     labels=['e11', 'e12', 'e21', 'e22'])
plt.show()

In [ ]:
print("Cramer's association coefficient: {:.3f}".format(chain['V'].mean()))
# 1.0 - 0.5: strong association
# 0.5 - 0.25: association
# 0.25 - 0.1: weak association
# 0.1 > : very weak association
# 0: no association

# Weak association has found, thus there is not much association between watching CM and purchasing the product.

In [ ]:
egz = pd.DataFrame(
    [[(chain['e'][:,0] > 0).mean(), (chain['e'][:,1] > 0).mean()],
     [(chain['e'][:,2] > 0).mean(), (chain['e'][:,3] > 0).mean()]],
     columns=columns, index=index
)

elz = pd.DataFrame(
    [[(chain['e'][:,0] < 0).mean(), (chain['e'][:,1] < 0).mean()],
     [(chain['e'][:,2] < 0).mean(), (chain['e'][:,3] < 0).mean()]],
     columns=columns, index=index
)

print('e > 0')
display(egz)

print('e < 0')
display(elz)